In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import pandas as pd
import numpy as np

In [6]:
from market_value_predictor.preproc import manual_encoding, reduce_number_of_classes

In [7]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, RobustScaler, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn import set_config; set_config(display='diagram')
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from xgboost.sklearn import XGBRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from catboost import CatBoostRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score

In [8]:
df = pd.read_csv("../../raw_data/master_df_with_webscraping.csv")
df_train = pd.read_csv("../../raw_data/master_data_train.csv").drop(columns="Unnamed: 0")
df_test = pd.read_csv("../../raw_data/master_data_test.csv").drop(columns="Unnamed: 0")

# Preprocessing

In [9]:
from sklearn.preprocessing import FunctionTransformer

drop_nas = FunctionTransformer(lambda df: df.dropna(subset=list(df.select_dtypes(object).columns)))

feat_eng_player_traits = FunctionTransformer(lambda df: manual_encoding(df, "player_traits"))
feat_eng_player_tags = FunctionTransformer(lambda df: manual_encoding(df, "player_tags"))
feat_eng_player_positions = FunctionTransformer(lambda df: manual_encoding(df, "player_positions"))

dim_reduction_nationality = FunctionTransformer(lambda df: reduce_number_of_classes(df, "nationality", 50))
dim_reduction_league_name = FunctionTransformer(lambda df: reduce_number_of_classes(df, "league_name", 100))

cluster_team_position = FunctionTransformer(lambda df: cluster_team_position(df))

feat_eng = Pipeline([
    ("player_traits", feat_eng_player_traits),
    ("player_tags", feat_eng_player_tags),
    ("player_positions", feat_eng_player_positions),
    ("nationality", dim_reduction_nationality),
    ("league_name", dim_reduction_league_name),
    ("drop_nas", drop_nas),
#     ("team_position", cluster_team_position)
])

feat_eng

Pipeline(steps=[('player_traits',
                 FunctionTransformer(func=<function <lambda> at 0x7fab572dbb80>)),
                ('player_tags',
                 FunctionTransformer(func=<function <lambda> at 0x7fab572dbc10>)),
                ('player_positions',
                 FunctionTransformer(func=<function <lambda> at 0x7fab572dbe50>)),
                ('nationality',
                 FunctionTransformer(func=<function <lambda> at 0x7fab572dbee0>)),
                ('league_name',
                 FunctionTransformer(func=<function <lambda> at 0x7fab572dbf70>)),
                ('drop_nas',
                 FunctionTransformer(func=<function <lambda> at 0x7fab572db940>))])

In [10]:
df_train_transformed = feat_eng.fit_transform(df_train)

In [11]:
df_test_transformed = feat_eng.transform(df_test)

# Pipeline

## Define lists of columns for pipeline

In [12]:
all_cats = list(df_train.select_dtypes(object).columns)

numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

all_numerics = list(df_train.select_dtypes(include=numerics).columns)

all_numerics.remove("fee_cleaned")

encoded_columns = [elem for elem in all_numerics if "player_tags_" in elem] + [
    elem for elem in all_numerics if "player_positions_" in elem
] + [elem for elem in all_numerics if "player_traits_" in elem]

all_numerics_wo_encoded = []
for elem in all_numerics:
    if elem not in encoded_columns:
        all_numerics_wo_encoded.append(elem)

numericals_zero_impute = [
    "gk_diving", "gk_handling", "gk_kicking", "gk_reflexes", "gk_speed",
    "gk_positioning", "release_clause_eur"
]

numericals_mean_impute = []

for elem in all_numerics_wo_encoded:
    if elem not in numericals_zero_impute:
        numericals_mean_impute.append(elem)

## Define transformers

In [13]:
from tempfile import mkdtemp
from shutil import rmtree
cachedir = mkdtemp()

In [14]:
num_zero_tr = Pipeline([
    ("imputer", SimpleImputer(strategy="constant", fill_value=0)),
    ("scaler", StandardScaler())
])

num_mean_tr = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler())
])

cat_tr = OneHotEncoder(handle_unknown='ignore')

## Build pipeline

In [15]:
preprocessor = ColumnTransformer(
    [("numerics_zero_imputing", num_mean_tr, numericals_zero_impute),
     ("numerics_mean_imputing", num_mean_tr, numericals_mean_impute),
     ("cat_tr", cat_tr, all_cats)],
remainder="passthrough")

pipe = Pipeline([
    #("feat_eng", feat_eng),
    ("preprocessing", preprocessor), 
    ("regressor", XGBRegressor())], memory=cachedir)

pipe

Pipeline(memory='/tmp/tmpq78ot3gn',
         steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numerics_zero_imputing',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['gk_diving', 'gk_handling',
                                                   'gk_kicking', 'gk_reflexes',
                                                   'gk_speed', 'gk_positioning',
                                                   'release_clause_eur']),
                                                 ('numerics_mean_imputing',
                                                  Pipe...
                              importance_type='gain',
                              interaction_constraints=None, learning_rate=None,
                              max_delta_step=None, max_depth=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, n_estimators=100,
                              n_jobs=None, num_parallel_tree=None,
                              random_state=None, reg_alpha=None,
                              reg_lambda=None, scale_pos_weight=None,
                              subsample=None, tree_method=None,
                              validate_parameters=None, verbosity=None))])

## Train pipeline    

In [16]:
X_train = df_train.drop(columns="fee_cleaned")
y_train = df_train[["fee_cleaned"]]
X_test = df_test.drop(columns="fee_cleaned")
y_test = df_test[["fee_cleaned"]]

In [17]:
pipe.fit(X_train, y_train)

Pipeline(memory='/tmp/tmpq78ot3gn',
         steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numerics_zero_imputing',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['gk_diving', 'gk_handling',
                                                   'gk_kicking', 'gk_reflexes',
                                                   'gk_speed', 'gk_positioning',
                                                   'release_clause_eur']),
                                                 ('numerics_mean_imputing',
                                                  Pipe...
                              colsample_bytree=1, gamma=0, gpu_id=-1,
                              importance_type='gain',
                              interaction_constraints='',
                              learning_rate=0.300000012, max_delta_step=0,
                              max_depth=6, min_child_weight=1, missing=nan,
                              monotone_constraints='()', n_estimators=100,
                              n_jobs=8, num_parallel_tree=1, random_state=0,
                              reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                              subsample=1, tree_method='exact',
                              validate_parameters=1, verbosity=None))])

In [18]:
y_pred = pipe.predict(X_test)

In [19]:
r2_score(y_test, y_pred)

0.40555763627147656

## Grid search

In [20]:
X = df.drop(columns="fee_cleaned")
y = df[["fee_cleaned"]]

In [21]:
cross_val_score(pipe, X, y, cv=5, scoring='r2').mean()

0.35145848188803974

In [22]:
pipe.get_params()

{'memory': '/tmp/tmpq78ot3gn',
 'steps': [('preprocessing',
   ColumnTransformer(remainder='passthrough',
                     transformers=[('numerics_zero_imputing',
                                    Pipeline(steps=[('imputer', SimpleImputer()),
                                                    ('scaler', StandardScaler())]),
                                    ['gk_diving', 'gk_handling', 'gk_kicking',
                                     'gk_reflexes', 'gk_speed', 'gk_positioning',
                                     'release_clause_eur']),
                                   ('numerics_mean_imputing',
                                    Pipeline(steps=[('imputer', SimpleImputer()),
                                                    ('scaler', StandardS...
                                     'skill_curve', 'skill_fk_accuracy',
                                     'skill_long_passing', 'skill_ball_control',
                                     'movement_acceleration',
        

In [23]:
param_grid = {
    "preprocessing__numerics_zero_imputing__scaler": [RobustScaler(), MinMaxScaler(), StandardScaler()],
    "preprocessing__numerics_mean_imputing__scaler": [RobustScaler(), MinMaxScaler(), StandardScaler()],
    'preprocessing__numerics_zero_imputing__imputer__strategy': ['constant', "mean"],
    'regressor__alpha': [0.01, 0.1, 1], 
    'regressor__l1_ratio': [0.2, 0.5, 0.8]
}

In [ ]:
# grid_search = GridSearchCV(
#     pipe, 
#     param_grid=param_grid,
#     cv=5,
#     scoring="r2",
#     n_jobs=-1)

# grid_search.fit(X, y)
# print(grid_search.best_params_)
# print(grid_search.best_score_)